## Part 2a - Sentiment Analysis

In [ ]:
# Import Packages
import nltk
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

### Extracting the 100 csv's with the scraped Yelp data from a zipped file  and loading them into Pandas dataframe

Please change the directory to the zip file location

In [ ]:
import os
# os.chdir("E:\\Purdue\\Fall 2\\Web Analytics\\Final Project")

In [2]:
import glob
import shutil
import pandas as pd

#Unzipped folder name
data1 = "restaurant_data"
path = os.getcwd()+"\\"+data1

#Unzip files from zip file
shutil.unpack_archive(os.getcwd()+"\\restaurant_data.zip",path)

fileNames=os.listdir(path) 

#Store all paths of the files using glob
for fp in fileNames:
    ext = os.path.splitext(fp)[-1].lower()
    files = glob.glob(os.path.join(path, "*"+ext))

#Concatenate and store all files in a dataframe
dfs = (pd.read_csv(f, engine='python', encoding = 'utf-8') for f in files)
final_df = pd.concat(dfs, ignore_index=True)

### Reading the pandas dataframe and renaming the columns

In [3]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np


#Renaming the column names
final_df.rename(columns = {'Name':'restaurant', 'Rating':'Rating', 
                              'Review':'review_text'}, inplace = True) 

sample_reviews=final_df



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# !pip install textblob

### Using TextBlob for sentiment analysis

In [5]:
from textblob import TextBlob

#Initializing 3 lists to hold the restaurant name, flag(live music or not) and the sentiment score of the review
restaurant_list_tblob=[]
flag_list_tblob=[]
sentiment_score_list_tblob=[]

#Iterate through all the review and find the sentiment score
for i in range(len(sample_reviews)):
    testimonial = TextBlob(sample_reviews.review_text[i])
    
    #Appending the restaurant name, flag(live music or not) and the sentiment score of the review into the lists initialized 
    restaurant_list_tblob.append(sample_reviews.restaurant[i])

    #flag_list_tblob.append(sample_reviews.music_flag[i])
    sentiment_score_list_tblob.append(testimonial.polarity)
    

### Loading the sentiment analysis for each review into a pandas dataframe and later aggregating at restaurant and flag (live-music bar or not) level

In [6]:
#Create a pandas dataframe with the 3 lists created in the previous snippet
sentiment_combined_textblob = pd.DataFrame(
    {'Restaurant': restaurant_list_tblob,
     'Sentiment_score': sentiment_score_list_tblob
    })

In [7]:
#Find the average sentiment score of all reviews aggregated at restaurant and music flag(live music or not)

sentiment_data=sentiment_combined_textblob.groupby(["Restaurant"])["Sentiment_score"].mean().reset_index()

In [8]:
#export the aggregated average sentiment scores for each restaurant into a csv file
sentiment_data.to_csv("restaurant_senti_score_total.csv",encoding = 'utf-8',index=False)

## Part 2b - Data Preparation for regression

In [9]:
# Read data for restaurant name and URLs
restaurants = pd.read_excel("restaurants_2.xlsx")

# Store Restaurant URLs and names in lists
rest_links = restaurants['Restaurant_URL'].tolist()
rest_names = restaurants['Restaurant_Name'].tolist()

In [17]:
# Merge restaurant name and average sentiment score data

new_df =pd.merge(restaurants, sentiment_data,left_on='Restaurant_Name',right_on = 'Restaurant',how='left')
new_df.drop(['Restaurant','Restaurant_URL'],axis = 1,inplace = True)

In [18]:
# Calculate average rating for each restaurant

x = final_df.groupby('restaurant')
x2 = x[['Rating']].mean()
x2.reset_index(inplace = True)

In [19]:
# Merge restaurant name and average rating data

final_df_2 = pd.merge(new_df, x2,left_on='Restaurant_Name',right_on = 'restaurant',how='left')
final_df_2.drop(['restaurant'],axis = 1,inplace = True)

In [20]:
# Set column names
final_df_2.columns = ['Restaurant_Name', 'Dummy_Live','High_Cost','Avg_Sent_Score','Avg_Rating']

In [21]:
# Calculate number of reviews for each restaurant

x3 = final_df.groupby('restaurant')
x4 = x3[['restaurant']].count()
x4.columns = ['Review_count']
x4.reset_index(inplace = True)


In [22]:
# Merge restaurant name and number of reviews data

final_df_3 = pd.merge(final_df_2, x4,left_on='Restaurant_Name',right_on = 'restaurant',how='left')
final_df_3.drop(['restaurant'],axis = 1,inplace = True)

In [23]:
# Write data as csv to perform regression in R

final_df_3.to_csv("Final_data.csv",encoding = 'utf-8', index = False)